### RDKit's capabilities at handling 3D molecules 

In [1]:
from rdkit.Chem import Descriptors
from rdkit import Chem 
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole 
from rdkit.Chem import Descriptors 
from rdkit.Chem import AllChem 
from rdkit import DataStructs
import numpy as np 
from rdkit.Chem import MolFromInchi
from rdkit.Chem import rdMolDescriptors
import pubchempy

#### Processing Molecule Graphs 
Given a SMILES String(canonical or noncanonical), RDKit can process the bonding information to get all atoms and the edges between the atoms. This is used at processing molecule datasets to get graph information 

In [ ]:
# RDKit automatially assigns the index to each atom 
mol= Chem.MolFromSmiles("c1ccccc1") # benzene C6H6 
for atom in mol.GetAtoms(): 
    print(f"Atom:{atom.GetIdx()} | Symbol:{atom.GetSymbol()}")

Atom:0 | Symbol:C
Atom:1 | Symbol:C
Atom:2 | Symbol:C
Atom:3 | Symbol:C
Atom:4 | Symbol:C
Atom:5 | Symbol:C


In [ ]:
print(mol.GetNumAtoms())    # 

6

### Incomplete Graph Representations 